In [110]:
from openpyxl import load_workbook


In [20]:
import os
os.system("open exercises/data/f1-results.xlsx")

0

Docs: https://realpython.com/openpyxl-excel-spreadsheets-python/

**Zadanie 1:** Przygotuj arkusz o nazwie *Standings*, który będzie zawierał sumaryczną liczbę punktów zawodników po każdym wyścigu. 
    
Arkusz powinien zawierać tabelą jak poniżej:

|                  | Australia | China | ... |
|------------------|-----------|-------|-----|
| Lewis Hamilton   | 18        | 43    | ... |
| Sebastian Vettel | 25        | 43    | ... |
| ...              |           |       |     |


In [108]:
# Zadanie 1a - wersja bez sumarycznej liczby punktow

wb = load_workbook('exercises/data/f1-results.xlsx')

# Przygotuj dane
drivers = {}
for sheet in wb:
    for row in sheet.iter_rows():
        name = row[2].value # column 3 in xlsx
        points = row[5].value # column 6 in xlsx
        drivers.setdefault(name, {})
        races = drivers[name]
        races.setdefault(sheet.title, 0)
        races[sheet.title] = points
        
#print(drivers)

# Zapisz dane do pliku xlsx

# Przygoduj wiersze tabeli
from openpyxl import Workbook
ws = wb.create_sheet("Standings")

data = []
for key, value in drivers.items():
    #print(key)
    row = [key]
    for race, points in value.items():
        row.append(int(points))
    #print(row)
    data.append(row)
    
# Dodaj nazwy kolumn do tabelki (nie uwzgledniaj karty "Standings")
ws.append([''] + wb.sheetnames[:-1])

# Mini formatowanie wygladu
from openpyxl.styles import Font, Color, Alignment, Border, Side, colors
bold_font = Font(bold=True)
for column in ws.iter_cols(min_col=2, min_row=1, max_col=21, max_row=1):
    for cell in column:
        #print(cell.font)
        cell.font = bold_font

# Nie mozna zaaplikowac stylu do zakresu komorek:
# https://stackoverflow.com/questions/37632327/openpyxl-assign-value-or-apply-format-to-a-range-of-excel-cells-without-iterati
    
# Dodaj wiersze do tabelki
for row in data:
    ws.append(row)

# Zapisz do pliku
wb.save('./exercises/tmp/f1-results-standings-noncumulative.xlsx')

In [ ]:
import os
os.system("open exercises/tmp/f1-results-standings-noncumulative.xlsx")

In [111]:
# Zadanie 1b - wersja z sumaryczna liczba punktow

wb = load_workbook('exercises/data/f1-results.xlsx')

# Przygotuj dane
drivers = {}
for sheet in wb:
    for row in sheet.iter_rows():
        name = row[2].value # column 3 in xlsx
        points = row[5].value # column 6 in xlsx
        drivers.setdefault(name, [])
        races = drivers[name]
        if len(races) == 0:
            cum_points = int(points)
        else:
            cum_points = races[-1] + int(points)
        races.append(cum_points)
        drivers[name] = races
        
#print(drivers)

# Zapisz dane do pliku xlsx

# Przygoduj wiersze tabeli
from openpyxl import Workbook
ws = wb.create_sheet("Standings")

data = []
for key, value in drivers.items():
    data.append([key] + value)
    
# Dodaj nazwy kolumn do tabelki (nie uwzgledniaj karty "Standings")
ws.append([''] + wb.sheetnames[:-1])

# Mini formatowanie wygladu
from openpyxl.styles import Font, Color, Alignment, Border, Side, colors
bold_font = Font(bold=True)
for column in ws.iter_cols(min_col=2, min_row=1, max_col=21, max_row=1):
    for cell in column:
        #print(cell.font)
        cell.font = bold_font

# Nie mozna zaaplikowac stylu do zakresu komorek:
# https://stackoverflow.com/questions/37632327/openpyxl-assign-value-or-apply-format-to-a-range-of-excel-cells-without-iterati
    
# Dodaj wiersze do tabelki
for row in data:
    ws.append(row)

# Zapisz do pliku
wb.save('./exercises/tmp/f1-results-standings-cumulative.xlsx')

In [112]:
import os
os.system("open exercises/tmp/f1-results-standings-cumulative.xlsx")

0

**Zadanie 2:** Na podstawie powyższej tabeli, w arkuszu *Standings* umieść wykres liniowy (`LineChart`), który zaprezentuje sumaryczną liczbę punktów po kolejnych wyścigach. Na osi X powinny być nazwy wyścigów, na osi Y liczba punktów, a serie reprezentują poszczególnych kierowców. 

In [ ]:
# Zadanie 2a - non-cumulative

from openpyxl import Workbook, load_workbook
from openpyxl.chart import (
    LineChart,
    Reference,
    Series
)

wb = load_workbook('exercises/tmp/f1-results-standings-noncumulative.xlsx')
ws = wb['Standings']

c1 = LineChart()
c1.title = "Non-cumulative points per race per driver"
#c1.style = 13
c1.y_axis.title = 'Points'
c1.x_axis.title = 'Race'

i = 2
for row in ws.iter_rows(min_row=2):
    #print(row[0].value) # kolumna 1 w arkuszu

    data = Reference(ws, min_row=i, min_col=2, max_row=i, max_col=21)
    serie = Series(data, title=row[0].value)
    c1.append(serie)
    
    i += 1

data = Reference(ws, min_row=1, min_col=2, max_row=1, max_col=21)
c1.set_categories(data)

c1.height = 15 # default is 7.5
c1.width = 30 # default is 15

ws.add_chart(c1, "K5")

wb.save("./exercises/tmp/f1-results-standings-chart-noncumulative.xlsx")

In [ ]:
import os
os.system("open exercises/tmp/f1-results-standings-chart-noncumulative.xlsx")

In [116]:
# Zadanie 2b - cumulative

from openpyxl import Workbook, load_workbook
from openpyxl.chart import (
    LineChart,
    Reference,
    Series
)

wb = load_workbook('exercises/tmp/f1-results-standings-cumulative.xlsx')
ws = wb['Standings']

c1 = LineChart()
c1.title = "Cumulative points per race per driver"
#c1.style = 13
c1.y_axis.title = 'Points'
c1.x_axis.title = 'Race'

i = 2
for row in ws.iter_rows(min_row=2):
    #print(row[0].value) # kolumna 1 w arkuszu

    data = Reference(ws, min_row=i, min_col=2, max_row=i, max_col=21)
    serie = Series(data, title=row[0].value)
    c1.append(serie)
    
    i += 1

data = Reference(ws, min_row=1, min_col=2, max_row=1, max_col=21)
c1.set_categories(data)

c1.height = 15 # default is 7.5
c1.width = 30 # default is 15

ws.add_chart(c1, "K5")

wb.save("./exercises/tmp/f1-results-standings-chart-cumulative.xlsx")

In [117]:
import os
os.system("open exercises/tmp/f1-results-standings-chart-cumulative.xlsx")

0